## Executing randomized measurement on IBM quantum computers

This tutorial illustrates how to acquire and save randomized measurements on a quantum computer using Qiskit (RandomizedMeasurementsQiskit.ipynb). This data can be then postprocessed using our julia library RandomMeas (RandomizedMeasurementsQiskitPostprocessing.ipynb)

In [11]:
import numpy as np
import random
from qiskit import QuantumCircuit, transpile
# from qiskit.visualization import plot_histogram
# from qiskit.circuit.library import  MCXGate
# from qiskit.quantum_info import Statevector
# from qiskit_algorithms import AmplificationProblem
from qiskit_aer import AerSimulator
from qiskit_ibm_provider import IBMProvider,least_busy 
from qiskit.quantum_info import random_unitary
from qiskit.circuit.library import UnitaryGate
# from tqdm import tqdm


### Parameters

We consider here only two qubits, and will generate randomized measurements for the purpose of measuring state purities

In [2]:
N = 2 #number of qubits
Nu = 100 #Number of random unitaries
NM = 1000 # number of shots

In [3]:
#Uncomment if you want to use the simulator
#backend = AerSimulator()

### Selection of the quantum computer

In [4]:
#IBMProvider.save_account(token='',overwrite=True)
provider = IBMProvider(instance='ibm-q/open/main')
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 and 
                                   not x.configuration().simulator and x.status().operational==True))
print("Chosen quantum computer ", backend)

Chosen quantum computer  <IBMBackend('ibm_kyoto')>


### Parametrization of the quantum circuit

We prepare a GHZ state on two qubits, and include the random unitaries in the definition of the quantum circuit. For consitency, we keep the same layout of physical qubits for all used random unitaries.

In [7]:
transpiled_circuits = []
u = 1j*np.zeros((Nu,N,2,2))
for r in range(Nu):
    circuit = QuantumCircuit(N)
    circuit.h(0)
    circuit.cx(0,1)
    for i in range(N):
        u[r,i,:,:] = random_unitary(2)
        circuit.append(UnitaryGate(u[r,i,:,:]), [i])
    circuit.measure_all()
    if r==0:
        transpiled_circuits.append(transpile(circuit, backend,optimization_level=3))
    else:
        transpiled_circuits.append(transpile(circuit, backend,optimization_level=3,initial_layout=transpiled_circuits[0].layout.final_virtual_layout()))

### Sending the job on IBM's hardware

In [8]:
job = backend.run(transpiled_circuits, shots=NM,memory=True)
print(job.queue_info())

### Saving data
Finally we convert the data structure used by qiskit to a numpy array that we will be able to load in julia later using the library NPZ

In [10]:
#job = provider.retrieve_job('') #uncomment if you want to retrieve a given job using its ID
data = np.zeros((Nu,NM,N),dtype=int)
result = job.result()
for r in range(Nu):
    counts = result.get_memory(r)
    for m in range(NM):
        data[r,m,:] = np.array([int(counts[m][N-1-i])+1 for i in range(N)])
np.savez("testQiskit.npz",u=u,data=np.int64(data))